In [1]:
# Recommended import aliases: https://spikeinterface.readthedocs.io/en/latest/getting_started/plot_getting_started.html
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
# Additional imports
import os.path as op
import neo  # neo for some direct reading to check on # of segments etc
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
#Get manual clustering results using Hung-tu's code
from loader_mclust import load_spikes
file_path = 'data/BiconditionalOdor/M040-2020-04-29-CDOD12/'
units = load_spikes(file_path)
sampling_frequency = 30000

# Let's concatenate all the units with unique per each unit labels
# into numpy arrays and pass it all to NumpySortingExtractor
all_times = []
all_labels = []
units_map = []
for u_index, u in enumerate(units):
    all_times += [u.time]
    all_labels += [u_index] * len(u.time)
    units_map.append(u.label)
all_times = np.hstack(all_times)
all_labels = np.hstack(all_labels)
assert all_times.ndim == 1
assert all_times.shape == all_labels.shape

mclust_res = se.NumpySortingExtractor()
mclust_res.set_times_labels(times=all_times, labels=all_labels )
mclust_res.set_sampling_frequency(sampling_frequency=sampling_frequency)

In [3]:
# Getting all the results directory
results_top_dir = 'data/Kilosort2_results/M040-2020-04-29-CDOD12/'
subdirs = [os.path.join(results_top_dir, dirname) for dirname in os.listdir(results_top_dir) if os.path.isdir(os.path.join(results_top_dir,dirname))]

In [4]:
# Looping through results to see if at least one unit matches
first_pass_candidates = {}
funny_business = {}
max_agreement_scores = {}

for dirname in subdirs:
    # Using PhySortingExtractor because Kilosort2 results are exported in the Phy format
    this_key = "_".join(dirname.split("_")[-2:])
    this_ksr = se.PhySortingExtractor(folder_path=dirname)
    this_com = sc.compare_two_sorters(sorting1=this_ksr, sorting2=mclust_res)
    #Saving this to see what kind of values pop out
    max_agreement_scores[this_key] = this_com.agreement_scores.max()
    # Check if even 1 unit matches, if it does add the comparsion and the sorting to the first_pass_candidates dictionary
    this_ks_map = this_com.get_mapped_sorting1()
    matches = [x for x in this_ks_map.get_mapped_unit_ids() if x != -1]
    if len(matches) == 0: # no matches found
        #this is just a sanity check, probably unnecessary
        matches2 = [x for x in this_com.get_mapped_sorting2().get_mapped_unit_ids() if x != -1]
        if len(matches2) != 0:
            #append to funny_business
            print(this_key, " : Funny business!\n")
            funny_business[this_key] = [this_ksr, this_com]
    else:
        print(this_key, " Number of matched units: ", len(matches))
        first_pass_candidates[this_key] = [this_ksr, this_com]

In [15]:
# Really low agreement score even for the max value
# No matches found from the above iteration
max_max = [(a, max_agreement_scores[a].max()) for a in max_agreement_scores]
# Need to investigate what's happening

In [16]:
# fig,ax = plt.subplots()
# st  = mclust_res.get_unit_spike_train(8)
# ax.plot(st, np.zeros(st.size), '|')
# st2 = this_ksr.get_unit_spike_train(34)
# ax.plot(st2, np.ones(st2.size), '|')

In [17]:
# ksr_1.get_unit_property_names()
# uids = ksr_1.get_unit_ids()
# q = [ksr_1.get_unit_property(x, 'KSLabel') for x in uids]
# len(ksr_1.get_units_property(property_name='KSLabel'))

In [18]:
# # units matched to ks2 units
# ksort1_map = com4.get_mapped_sorting1()
# print('Ksort2 units:', ksr_1.get_unit_ids())
# print('Ksort2 mapped units:', ksort1_map.get_mapped_unit_ids())

# # units matched to msort units
# mclust_map = com4.get_mapped_sorting2()
# print('Mclust units:', msort.get_unit_ids())
# print('Mclust mapped units:', mclust_map.get_mapped_unit_ids())